# SPoOkY Meshes 👻

In [40]:
import base64
import time

import numpy as np
import scipy as sp
import scipy.sparse
import scipy.sparse.linalg
import pymeshlab
import k3d
import IPython

In [41]:
def load_mesh(filename):
    ms = pymeshlab.MeshSet()
    ms.load_new_mesh(filename)
    v = ms.current_mesh().vertex_matrix()
    f = ms.current_mesh().face_matrix()
    return v, f


def save_mesh(filename, v, f):
    ms = pymeshlab.MeshSet()
    m  = pymeshlab.Mesh(vertex_matrix=vx, face_matrix=f)
    ms.add_mesh(m)
    ms.save_current_mesh(filename)


v1, f1 = load_mesh('cw2_meshes/curvatures/plane.obj')
v2, f2 = load_mesh('cw2_meshes/curvatures/lilium_s.obj')
v3, f3 = load_mesh('cw2_meshes/decompose/armadillo.obj')
v4, f4 = load_mesh('cw2_meshes/smoothing/fandisk_ns.obj')
v5, f5 = load_mesh('cw2_meshes/smoothing/plane_ns.obj')
v7, f7 = load_mesh('sappho-hires.obj')

In [42]:
def normalize(v):
    """
    recenter points
    """
    if not np.any(v):
        return v
    return 2 * (v - v.min()) / (v.max() - v.min()) - 1


def percentile_clip(h, n):
    """
    reject outlier points
    """
    return np.clip(h, np.percentile(h, n), np.percentile(h, 100 - n))


def plot(grid=False):
    """
    create a plot for visualization
    """
    plot = k3d.plot(grid_visible=grid)
    return plot


def plot_mesh(p, v, f, h, t=None, r=None, wf=False):
    m = k3d.mesh(v.astype(np.float32), f.astype(np.uint32), attribute=h, wireframe=wf)
    m.transform.translation = t
    m.transform.rotation = r
    p += m
    return m

In [51]:
# Unsexy test mesh for debugging
v0 = np.array([
    [ 0, 0, 0],
    [-3, 4, 0],
    [-6, 0, 0],
    [-3,-4, 0],
    [ 3,-4, 0],
    [ 6, 0, 0],
    [ 3, 4, 0],
], dtype=np.float32)

f0 = np.array([
    [0, 1, 2],
    [0, 2, 3],
    [0, 3, 4],
    [0, 4, 5],
    [0, 5, 6],
    [0, 1, 6],
], dtype=np.int32)

# perturbed vertex indices to throw off naive algorithms
# f0 = np.array([
#     [1, 0, 2],
#     [2, 3, 0],
#     [0, 3, 4],
#     [4, 0, 5],
#     [5, 6, 0],
#     [6, 1, 0],
# ], dtype=np.int32)

## Laplace-Beltrami and Friends

### Vertex Area

In [52]:
def triangle_area(v, f):
    i, j, k = f.T
    a, b, c = v[i], v[j], v[k]

    ac = c - a
    bc = c - b

    return np.linalg.norm(np.cross(ac, bc, axis=1), axis=1) / 2


def vertex_area(v, f):
    """
    compute total area about vertices
    3.59ms for 281,724 faces, not bad son
    """
    n = len(v)
    A = np.zeros((3, len(f)))

    area = triangle_area(v, f)

    # set internal angles at vertex location in face array
    # using indexes that have duplicate values to increment doesn't work
    A[0] = area
    A[1] = area
    A[2] = area

    # some esoteric numpy for summing at duplicated indices
    # coo matrices are also an option
    data = A.ravel()
    cols = f.T.ravel()

    M = np.zeros(n)
    np.add.at(M, cols, data)

    return sp.sparse.diags(M)


def barycentric_mass(v, f):
    return vertex_area(v, f) / 3


# %timeit vertex_area(v4, f4)


M = barycentric_mass(v0, f0)
assert list(M.diagonal()) == [24, 8, 8, 8, 8, 8, 8]

### Cotangent operator

In [53]:
def cotangent_curvature(v, f):
    """
    we can sum the contribution of all adjacent angles,
    then subtract the contribution of all interior angles.
    
    using edges would probably avoid some redundant work here (as above).

    133ms seconds for 281,724 faces. igl is 52.2ms.
    """
    n = len(v)
    
    # indices
    i, j, k = f.T
    a, b, c = v[i], v[j], v[k]

    # vectors
    ab = b - a
    ac = c - a
    bc = c - b
    
    # big chungus cotangent computation
    abc = np.einsum('ij,ij->i', ab, ac) / np.linalg.norm(np.cross( ab, ac, axis=1), axis=1)
    bac = np.einsum('ij,ij->i',-ab, bc) / np.linalg.norm(np.cross(-ab, bc, axis=1), axis=1)
    cab = np.einsum('ij,ij->i',-ac,-bc) / np.linalg.norm(np.cross(-ac,-bc, axis=1), axis=1)

    # set weights for opposite edges
    # a csr_matrix will sum the quantities for us!
    data = np.hstack([cab, abc, bac])
    rows = np.hstack([i,   j,   k, ])
    cols = np.hstack([j,   k,   i, ])
    T = sp.sparse.csr_matrix((data, (rows, cols)))

    # mad gains by flipping across the diagonal ;)
    T = T + T.T

    # Nearly there. sum the rows and use as diagonal.
    S = sp.sparse.diags(T.sum(axis=1).A.ravel())
    T = T - S

    # i'm never doing this again
    # divide by two as we have computed k_1 + k_2
    return T / 2


# %timeit cotangent_curvature(v7, f7)

C = cotangent_curvature(v0, f0)
T = np.diag(np.round((180 / np.pi) * C.todense()))

assert list(T) == [-205, -73, -60, -73, -73, -60, -73]
assert np.sum(C) < 1e-6

## TV Divergence & Gradient Operators

hack this together

$$
\mathbf{G} f\left(\mathbf{t}_{i j k}\right)=\left(\begin{array}{cc}
\mathbf{v}_{j}^{\top}-\mathbf{v}_{i}^{\top} \\
\mathbf{v}_{k}^{\top}-\mathbf{v}_{i}^{\top}
\end{array}\right)^{\top}\left(\begin{array}{cc}
\left\|e_{i j}\right\|^{2} & \left\langle e_{i j}, e_{i k}\right\rangle \\
\left\langle e_{i j}, e_{i k}\right\rangle & \left\|e_{i k}\right\|^{2}
\end{array}\right)\left(\begin{array}{l}
f\left(\mathbf{v}_{j}\right)-f\left(\mathbf{v}_{i}\right) \\
f\left(\mathbf{v}_{k}\right)-f\left(\mathbf{v}_{i}\right)
\end{array}\right)
$$

In [54]:
def tv_gradient_operator(v, f, axis=0):
    # indices
    i, j, k = f.T

    # edges?
    e_ij = v[j] - v[i]
    e_ik = v[k] - v[i]

    # hstack and reshape into F x 2 x 3
    # TODO: can we remove this swapaxes?
    A = np.hstack([
        e_ij,
        e_ik,
    ]).reshape(len(f), 2, 3).swapaxes(1, 2)

    e_ij2 = np.sum(e_ij ** 2, axis=1)
    e_ik2 = np.sum(e_ik ** 2, axis=1)

    e_ijik = np.einsum('ij,ij->i', e_ij, e_ik)

    B = np.hstack([
        [e_ij2, e_ijik, e_ijik, e_ik2],             
    ]).reshape(2, 2, len(f)).T

    # our 'function' is the X coordinate for now
    C = np.array([
        v[j][:, axis] - v[i][:, axis],
        v[k][:, axis] - v[i][:, axis],
    ]).reshape(len(f), 2, 1)

    D = A @ B @ C
    
    rows = np.repeat(np.arange(0, len(f)), 3)
    cols = f.ravel()
    data = D.ravel()

    G = sp.sparse.csr_matrix((data, (rows, cols)))
    return G

# Test that our unsexy mesh produces the correct value for the first row
assert np.allclose(tv_gradient_operator(v0, f0).todense().A[0], [2169, -732, 0, 0, 0, 0, 0])

In [57]:
def mean_curvature_metric(v, f):
    M = barycentric_mass(v, f)
    C = cotangent_curvature(v, f)
    Mi = sp.sparse.diags(1 / M.diagonal())
    Hn = -Mi @ C @ v
    H  = np.linalg.norm(Hn, axis=1)
    return H

def tv_gradient_metric(v, f, a):
    G = tv_gradient_operator(v, f, a)
    A = barycentric_mass(v, f)

    T = sp.sparse.diags(triangle_area(v, f))

    Ai = sp.sparse.diags(1/A.diagonal())

    D = -Ai @ G.T @ T
    
    H = scipy.sparse.linalg.norm(D, axis=1)
    return H

h1 = normalize(percentile_clip(tv_gradient_metric(v7, f7, 0), 5))
h2 = normalize(percentile_clip(tv_gradient_metric(v7, f7, 1), 5))
h3 = normalize(percentile_clip(tv_gradient_metric(v7, f7, 2), 5))

p = plot()
plot_mesh(p, v7, f7, h1, t=[-105, 0, 0])
plot_mesh(p, v7, f7, h2, t=[-45, 0, 0])
plot_mesh(p, v7, f7, h3, t=[15, 0, 0])

p.display()
p.camera = [0, 55, 700, 0, 55, 0, 0, 1, 0]
p.camera_fov = 10

Output()